# Updates

After updating packages, restart the session and then start your notebook from imports.

# Imports

In [48]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import pandas as pd
import re
import string
import spacy
from textblob import TextBlob

## Reading Json Files

In [28]:
def jsonl_tolist(jsonl_file_path):
    records = []
    with open(jsonl_file_path, 'r') as f:
        for line in f:
            record = json.loads(line)
            records.append(record)
    return records


In [29]:
jsonl_files_path = {
    "train": "imdb/train_imdb.jsonl",
    "test": "imdb/test_imdb.jsonl",
    "unlabeled": "imdb/aug_imdb_unlabeled.jsonl"
}

In [30]:
df_train = pd.DataFrame(jsonl_tolist(jsonl_files_path["train"]))
df_test = pd.DataFrame(jsonl_tolist(jsonl_files_path["test"]))
df_unlabeled = pd.DataFrame(jsonl_tolist(jsonl_files_path["unlabeled"]))

In [31]:
df_train.to_csv("train.csv")
df_test.to_csv("test.csv")
df_unlabeled.to_csv("unlabeled.csv")

In [32]:
def get_feature_columns_name(df):
    features_columns = []
    for i in range(len(df["embedding"][0])):
        name = "feature" + f"{i + 1}"
        features_columns.append(name)
    return features_columns

In [33]:
df_train[get_feature_columns_name(df_train)] = df_train["embedding"].apply(pd.Series)
df_test[get_feature_columns_name(df_test)] = df_test["embedding"].apply(pd.Series)
df_unlabeled[get_feature_columns_name(df_unlabeled)] = df_unlabeled["embedding"].apply(pd.Series)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_21532\1150993862.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train[get_feature_columns_name(df_train)] = df_train["embedding"].apply(pd.Series)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_21532\1150993862.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train[get_feature_columns_name(df_train)] = df_train["embedding"].apply(pd.Series)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_21532\1150993862.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usual

In [49]:
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(text, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)


In [50]:
def to_lowercase(text):
    return text.lower()

In [51]:
def remove_digits(text):
    return re.sub("\w*\d\w*", "", text)

In [52]:
def remove_punctuation(text):
    return re.sub("[%s]" % re.escape(string.punctuation), "", text)

In [53]:
def remove_extra_spaces(text):
    return re.sub(" +", " ", text)

In [54]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

negation_words = {"not", "no", "never", "none", "nobody", "nothing", "neither", "nowhere", "cannot"}

def lemmatize_and_remove_stop_words(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop or token.text.lower() in negation_words])

In [55]:
def calculate_polarity(text):
    return TextBlob(text).sentiment.polarity

In [56]:
def preprocess_dataframe(df):
    df = df.copy()
    df["text"] = df["text"].apply(expand_contractions)
    df["text"] = df["text"].apply(to_lowercase)
    df["text"] = df["text"].apply(remove_digits)
    df["text"] = df["text"].apply(remove_punctuation)
    df["text"] = df["text"].apply(remove_extra_spaces)
    df["text"] = df["text"].apply(lemmatize_and_remove_stop_words)
    df["polarity"] = df["text"].apply(calculate_polarity)
    return df

In [57]:
df_train = preprocess_dataframe(df_train)
df_test = preprocess_dataframe(df_test)
df_unlabeled = preprocess_dataframe(df_unlabeled)

# Load model

In [ ]:
MODEL_ARGS = {
    'Name': 'microsoft/Phi-3-mini-128k-instruct',
    'DType': bfloat16
}

In [ ]:
def load_model(model_args):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = AutoModelForCausalLM.from_pretrained(
        model_args['Name'],
        trust_remote_code=True,
        torch_dtype=torch.model_args['DType'],
        low_cpu_mem_usage=True,
        device_map={"": device},
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args['Name'],
        trust_remote_code=True,
    )

    return model, tokenizer

Good luck :)